# Modelling using XGBoost

Import modules

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('seaborn')

from geopy.distance import geodesic

from sklearn.model_selection import train_test_split
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor

from scipy import stats
from scipy.stats import randint

from prepare_flight_data import *
from feature_engineering import *

import pickle

RSEED = 42

Load data and prepare data

In [2]:
df, df_test = load_prepare_flight_data()
df.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,...,icao_ARR,iata_ARR,name_ARR,city_ARR,subd_ARR,country_ARR,elevation_ARR,lat_ARR,lon_ARR,tz_ARR
0,train_id_15674,2016-01-01,TU 0564,NKC,TUN,2016-01-01 00:15:00,2016-01-01 04:30:00,ATA,TU 320IMV,0.0,...,DTTA,TUN,Tunis Carthage International Airport,Tunis,Tunis,TN,22,36.851002,10.22720,Africa/Tunis
1,train_id_15676,2016-01-01,TU 0714,JED,TUN,2016-01-01 00:55:00,2016-01-01 05:30:00,ATA,TU 332IFM,195.0,...,DTTA,TUN,Tunis Carthage International Airport,Tunis,Tunis,TN,22,36.851002,10.22720,Africa/Tunis
2,train_id_15675,2016-01-01,TU 0614,DKR,TUN,2016-01-01 01:20:00,2016-01-01 05:55:00,ATA,TU 320IMU,49.0,...,DTTA,TUN,Tunis Carthage International Airport,Tunis,Tunis,TN,22,36.851002,10.22720,Africa/Tunis
3,train_id_30980,2016-01-01,UG 0002,TUN,DJE,2016-01-01 06:15:00,2016-01-01 07:15:00,SCH,UG AT7LBD,0.0,...,DTTJ,DJE,Djerba Zarzis International Airport,Djerba,Madanin,TN,19,33.875000,10.77550,Africa/Tunis
4,train_id_7179,2016-01-01,TU 0880,TUN,AMS,2016-01-01 06:30:00,2016-01-01 09:20:00,ATA,TU 736IOP,36.0,...,EHAM,AMS,Amsterdam Airport Schiphol,Amsterdam,North Holland,NL,-11,52.308601,4.76389,Europe/Amsterdam


Feature engineering

Runs around 15 seconds!

In [3]:
# Calculate distances
df = lat_lon_distance(df)
df_test = lat_lon_distance(df_test)
# Initialize custom transformer
#features_enable = [1, 1, 1, 1, 1] # [domestic, dep_hour, dep_weekday, duration_min, operator]
# Pipeline to add features
attr_addr = flight_preprocessor()
df = attr_addr.fit_transform(df)
df_test = attr_addr.transform(df_test)
# Store ID for submission
sub_id = df_test.ID
# Drop unimportant columns
cols_to_drop = ["icao_DEP", "iata_DEP", "name_DEP", "subd_DEP", "city_DEP", 
                "icao_ARR", "iata_ARR", "name_ARR", "subd_ARR", "city_ARR", 
                "ID", 'DATOP', "STA", "STD", 'tz_DEP', 'tz_ARR']
df = drop_column(df, cols_to_drop)
df_test = drop_column(df_test, cols_to_drop)

In [4]:
df.columns

Index(['FLTID', 'DEPSTN', 'ARRSTN', 'STATUS', 'AC', 'target', 'country_DEP',
       'elevation_DEP', 'lat_DEP', 'lon_DEP', 'country_ARR', 'elevation_ARR',
       'lat_ARR', 'lon_ARR', 'distance', 'domestic', 'dep_hour', 'dep_weekday',
       'duration_min', 'operator', 'arr_hour', 'dep_day', 'year'],
      dtype='object')

Train-test-split

In [5]:
# Train dataset
X = df.copy()
y = np.log(X.pop("target")+1)
# Test dataset for submission
X_submission = df_test
# Test-train-split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=RSEED)
print("Train size: ", y_train.shape, "\n" "Test size: ", y_test.shape)
# Remove outliers to improve zindi score
X_train = X_train[y_train < y_train.quantile(q=0.99)]
y_train = y_train[y_train < y_train.quantile(q=0.99)]
# X_test = X_test[y_test < y_train.quantile(q=0.99)]
# y_test = y_test[y_test < y_train.quantile(q=0.99)]
# print("Train size: ", y_train.shape, "\n" "Test size: ", y_test.shape)

Train size:  (97049,) 
Test size:  (10784,)


Preprocessing pipeline

Encoding of categorical features

In [6]:
categories_country_ARR = [np.asarray(df.groupby("country_ARR").mean().sort_values(by="target").index)]
categories_country_DEP = [np.asarray(df.groupby("country_DEP").mean().sort_values(by="target").index)]
categories_DEPSTN = [np.asarray(df.groupby("DEPSTN").mean().sort_values(by="target").index)]
categories_ARRSTN = [np.asarray(df.groupby("ARRSTN").mean().sort_values(by="target").index)]
categories_FLTID = [np.asarray(df.groupby("FLTID").mean().sort_values(by="target").index)]
categories_AC = [np.asarray(df.groupby("AC").mean().sort_values(by="target").index)]

Preprocessor pipelines

In [7]:
# Preprocessor pipelines
num_cols = ['distance', 'domestic', 'dep_hour', 'dep_weekday',
            'duration_min', "dep_day", "arr_hour"]
cat_cols = ["STATUS", "operator"]
# Preprocessor for numerical features
num_pipeline = Pipeline([
    #('num_scaler', StandardScaler()),
    ('num_scaler', MinMaxScaler())
])
# Preprocessor for categorical features 
cat_pipeline = Pipeline([
    ('cat_encoder', OneHotEncoder(handle_unknown='ignore'))
])
# Put together preprocessor pipeline
preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cat', cat_pipeline, cat_cols),
    ('cat_AC', OrdinalEncoder(categories=categories_AC, handle_unknown="use_encoded_value", 
                                unknown_value=(len(categories_AC[0])+1)), ["AC"]),
    ('cat_FLTID', OrdinalEncoder(categories=categories_FLTID, handle_unknown="use_encoded_value", 
                                unknown_value=(len(categories_FLTID[0])+1)), ["FLTID"]),
    ('cat_ARRSTN', OrdinalEncoder(categories=categories_ARRSTN, handle_unknown="use_encoded_value", 
                                unknown_value=(len(categories_ARRSTN[0])+1)), ["ARRSTN"]),
    ('cat_DEPSTN', OrdinalEncoder(categories=categories_DEPSTN, handle_unknown="use_encoded_value", 
                                unknown_value=(len(categories_DEPSTN[0])+1)), ["DEPSTN"]),
    ('cat_country_ARR', OrdinalEncoder(categories=categories_country_ARR, handle_unknown="use_encoded_value", 
                                unknown_value=(len(categories_country_ARR[0])+1)), ["country_ARR"]),
    ('cat_country_DEP', OrdinalEncoder(categories=categories_country_DEP, handle_unknown="use_encoded_value", 
                                unknown_value=(len(categories_country_DEP[0])+1)), ["country_DEP"])
])

Model pipeline

In [8]:
pipe_xgb = Pipeline([
    # ('attr_addr', features_pipeline),
    ('preprocessor', preprocessor),
    ('xgb', XGBRegressor())
])

In [9]:
# Defining parameter grid (as dictionary)
param_grid = {'xgb__n_estimators': list(range(10,100,50)),
              'xgb__learning_rate': list(np.linspace(0.001, 0.5, 100)),
              'xgb__subsample': [0.5],
              'xgb__gamma': [5],
              'xgb__max_depth': list(range(1, 10))
    }
# Final parameters
param_grid = {'xgb__n_estimators': [200],
              'xgb__learning_rate': [0.1],
              'xgb__subsample': [0.5],
              'xgb__gamma': [1, 2, 3],
              'xgb__max_depth': [10],
              'xgb__eta': [0.05]
    }
# Instantiate gridsearch and define the metric to optimize 
gs = RandomizedSearchCV(pipe_xgb,  param_grid, scoring='neg_root_mean_squared_error',
                  cv=6, verbose=0, n_jobs=-1, n_iter=20)

# Fit gridsearch object to data.
gs.fit(X_train, y_train)

RandomizedSearchCV(cv=6,
                   estimator=Pipeline(steps=[('preprocessor',
                                              ColumnTransformer(transformers=[('num',
                                                                               Pipeline(steps=[('num_scaler',
                                                                                                MinMaxScaler())]),
                                                                               ['distance',
                                                                                'domestic',
                                                                                'dep_hour',
                                                                                'dep_weekday',
                                                                                'duration_min',
                                                                                'dep_day',
                                            

Create linear targets and predictions

In [10]:
y_train_lin = np.exp(y_train) - 1
bl = y_train_lin.median()
y_pred_bl = [bl for el in X_test.index]
y_test_lin = np.exp(y_test) - 1
score = mean_squared_error(y_test_lin, y_pred_bl, squared=False)
print("RMSE for test dataset", score)

RMSE for test dataset 132.26709521647086


Output best estimator and scores

In [11]:
print(gs.best_estimator_)

# Making predictions on the train dataset
y_pred = gs.predict(X_train)
y_pred_lin = np.exp(y_pred) - 1
y_train_lin = np.exp(y_train) - 1
#y_pred = best_model.predict(X_train)
score = mean_squared_error(y_train_lin, y_pred_lin, squared=False)
print("RMSE for train dataset", score)

# Making predictions on the test dataset
y_pred = gs.predict(X_test)
y_pred_lin = np.exp(y_pred) - 1
y_test_lin = np.exp(y_test) - 1
score = mean_squared_error(y_test_lin, y_pred_lin, squared=False)
print("RMSE for test dataset", score)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('num_scaler',
                                                                   MinMaxScaler())]),
                                                  ['distance', 'domestic',
                                                   'dep_hour', 'dep_weekday',
                                                   'duration_min', 'dep_day',
                                                   'arr_hour']),
                                                 ('cat',
                                                  Pipeline(steps=[('cat_encoder',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['STATUS', 'operator']),
                                                 ('cat_AC',
                                                  OrdinalEncode

Export predictions for error analysis

In [12]:
df_X_test = pd.DataFrame(X_test)
df_y_test = pd.DataFrame(y_test)
df_y_pred = pd.DataFrame(y_pred)
df_X_test.to_csv("data/rf_X_test.csv", index=False)
df_y_test.to_csv("data/rf_y_test.csv", index=False)
df_y_pred.to_csv("data/rf_y_pred.csv", index=False)

Export submission for zindi

In [13]:
sub_target = np.exp(gs.predict(X_submission)) - 1

submission = pd.DataFrame(sub_id)
submission["target"] = sub_target
submission.to_csv("data/submission_rf1.csv", index=False)

Export model using pickle

In [14]:
filename = 'models/xgb_model.sav'
pickle.dump(gs.best_estimator_, open(filename, 'wb'))